# Build a Singularity container using AWS CodePipeline 

This notebook runs on conda_python3 kernel. 

In container-basic notebook, we used the command line to create, build and run a few containers on the same instance where we run the notebook. 

In this notebook, we will build a CICD pipeline to 
1. Build a docker container image and push it to ECR
1. Let the ECR scan for vulnerability in the pushed image
1. Manually approve the process after revieing the finding in ECR
1. Build the singularity container and push the singularity container image file (sif)  in S3
1. Pull down the zip of the sif file from pipeline output artifact folder and unzip it. 


In [ ]:
import boto3
import botocore
import json
import time
import os
import project_path # path to helper methods
import importlib
from lib import workshop
from botocore.exceptions import ClientError
from IPython.display import display, clear_output

# create a bucket for the workshop to store output files. 
session = boto3.session.Session()

region_name = session.region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

proj_name = 'sigularity-cicd-pipeline'
image_tag = 'mySRATools'

# we will use this bucket for some artifacts and the output of sratools. 
bucket = workshop.create_bucket(region_name, session, f"container-{proj_name}-{account_id}", False)
print(bucket)

In [ ]:
# same helper magic for Jupyter to create files more easily
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w+') as f:
        f.write(cell.format(**globals()))

# Automate the build process of SRA-Tools container

We will re-use the customized container from "Container Basic" notebook and continue the genomics use case with the NCBI SRA (Sequence Read Archive) SRA Tool (https://github.com/ncbi/sra-tools) and fasterq-dump (https://github.com/ncbi/sra-tools/wiki/HowTo:-fasterq-dump) to extract fastq from SRA-accessions.

The command takes a package name as an argument:
```
$ fasterq-dump SRR000001
```

We will use the base Ubuntu image and install sra-tools (https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/2.10.0/sratoolkit.2.10.0-ubuntu64.tar.gz) 

The workflow of the program in the contianer: 
1. Upon start, container runs a script "sratest.sh".
3. sratest.sh will "prefetch" the data package, whose name is passed via an environment variable. 
4. sratest.sh then run "fasterq-dump" on the data package
5. sratest.sh will then upload the result to S3://{bucket}

The output of the fasterq-dump will be stored in s3://{bucket}/data/sra-toolkit/fasterq/{PACKAGE_NAME}

## AWS CodePipeline

In the "Container Basics" notebook we built and ran the container on the same instance that runs the notebook. Here we will use AWS CodePipeline to automate a CI/CD process for our tools.

The AWS CodePipeline consists of the following components:

1. CodeCommit - code repository, which will contain a "buildspec.yml", "Dockerfile", and all files needed for the container.
2. CodeBuild - this will spawn an instance to run the "docker build" and "push" the image to Amazon ECR.
3. CodeDeploy - we will not use CodeDeploy in this notebook.

Each time we checkin code to CodeCommit, it will trigger the entire CodePipeline. When the pipeline finishes, we will have a new version of the docker image in the container registry (Amazon ECR). This allows downstream recipients of our image to take advantage of our work without having to understand the internal details of our container.

In [ ]:
PACKAGE_NAME='SRR000002'

# this is where the output will be stored
sra_prefix = 'data/sra-toolkit/fasterq'
sra_output = f"s3://{bucket}/{sra_prefix}"

print(sra_output)

### Step 1. Create the run script
This script will be executed via the RUN command in our container. 
1. It will fetch the sra package by package name
2. run fasterq-dump on the package data 
3. copy the output to S3

Note: for teaching purposes, we will be using a different syntax (""") in this notebook to prepare our files. See Step 4 for the context where we use them.

In [ ]:
sratest_content = """#!/bin/bash
set -x

prefetch $PACKAGE_NAME --output-directory /tmp
fasterq-dump $PACKAGE_NAME -e 8
aws s3 sync . $SRA_OUTPUT/$PACKAGE_NAME
"""

### Step 2. Create our own Docker image file

Let's build our own image using a Ubuntu base image. 

1. Install tzdata - as before, we will install this dependency explicitly (with -y argument) to avoid the timezone prompt that would halt the docker build process
2. Install wget and awscli.
3. Download sratookit ubuntu64 binary and unzip into /opt
4. Set the PATH to include the latest sratoolkit/bin
5. USER nobody is needed to set the permission for sratookit configuration. 
6. Use the same sratest.sh script 

Note: As of Nov 2020, DockerHub has set request limits on their public repos and you might get throttled if you use DockerHub's base image. Therefore, in this example, we will use the base Ubuntu image from AWS public docker registry.

In [ ]:
dockerfile_content="""FROM public.ecr.aws/ubuntu/ubuntu:latest

RUN apt-get update 
RUN DEBIAN_FRONTEND="noninteractive" apt-get -y install tzdata 
RUN apt-get install -y curl wget libxml-libxml-perl awscli uuid-runtime
        
RUN wget -q https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current-ubuntu64.tar.gz -O /tmp/sratoolkit.tar.gz \
        && tar zxf /tmp/sratoolkit.tar.gz -C /opt/ && rm /tmp/sratoolkit.tar.gz && \
        ln -s /opt/sratoolkit.$(curl -s "https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current.version")-ubuntu64 /opt/sratoolkit
        
ENV PATH="/opt/sratoolkit/bin/:${PATH}"
ADD sratest.sh /usr/local/bin/sratest.sh
RUN chmod +x /usr/local/bin/sratest.sh
RUN mkdir /tmp/.ncbi && printf '/LIBS/GUID = "%s"\\n' `uuidgen` > /tmp/.ncbi/user-settings.mkfg

ADD filelist.txt /tmp/filelist.txt
ENV HOME=/tmp
WORKDIR /tmp
USER nobody
ENTRYPOINT ["/usr/local/bin/sratest.sh"]
"""


### Step 3. Create the build spec file for the docker image build step
We will be using the AWS CodeBuild to create our docker image and store it into AWS ECR (private docker image registry). Thus, we need to create a build specifications for this service, such that it knows what commands to run to setup our build environment (pre_build), do the actual build (build section), and then push the image to AWS ECR (post_build). 

In [ ]:

buildspec_content ="""version: 0.2

phases:
  pre_build:
    commands:
      - echo Logging in to Amazon ECR...
      - aws ecr get-login-password --region $AWS_DEFAULT_REGION | docker login --username AWS --password-stdin $AWS_ACCOUNT_ID.dkr.ecr.$AWS_DEFAULT_REGION.amazonaws.com
  build:
    commands:
      - echo Build started on `date`
      - echo Building the Docker image...          
      - docker build -t $IMAGE_REPO_NAME:$IMAGE_TAG -f Dockerfile.cicd . 
      - docker tag $IMAGE_REPO_NAME:$IMAGE_TAG $AWS_ACCOUNT_ID.dkr.ecr.$AWS_DEFAULT_REGION.amazonaws.com/$IMAGE_REPO_NAME:$IMAGE_TAG      
  post_build:
    commands:
      - echo Build completed on `date`
      - echo Pushing the Docker image...
      - docker push $AWS_ACCOUNT_ID.dkr.ecr.$AWS_DEFAULT_REGION.amazonaws.com/$IMAGE_REPO_NAME:$IMAGE_TAG
"""

#place holder for later use , add in container so we don't have to change the docker file
file_list_content = """
"""

### Step 5. Create the build spec file for the singularity build step


In [ ]:
buildspec_content_sig ="""version: 0.2

phases:
  pre_build:
    commands:
      - echo Logging in to Amazon ECR...
      - aws ecr get-login-password --region $AWS_DEFAULT_REGION | docker login --username AWS --password-stdin $AWS_ACCOUNT_ID.dkr.ecr.$AWS_DEFAULT_REGION.amazonaws.com
  build:
    commands:
      - echo Build started on `date`
      - echo Install dependencies for singularity container 
      - yum groupinstall -y 'Development Tools'
      - yum install -y epel-release
      - yum install -y golang libseccomp-devel squashfs-tools cryptsetup wget git
      - wget -O go1.21.3.linux-amd64.tar.gz https://dl.google.com/go/go1.21.3.linux-amd64.tar.gz
      - tar -C /usr/local -xzf go1.21.3.linux-amd64.tar.gz
      - echo 'export GOPATH=${HOME}/go' >> ~/.bashrc
      - echo 'export PATH=/usr/local/go/bin:${PATH}:${GOPATH}/bin' >> ~/.bashrc
      - source ~/.bashrc
      - mkdir -p ${GOPATH}/src/github.com/apptainer
      - cd ${GOPATH}/src/github.com/apptainer
      - git clone --recurse-submodules https://github.com/apptainer/singularity.git
      - cd singularity
      - git checkout v3.8.7
      - ./mconfig 
      - cd ./builddir
      - make
      - make install
      - SINGULARITY_NOHTTPS=1 singularity build /tmp/$IMAGE_TAG.sif docker://$AWS_ACCOUNT_ID.dkr.ecr.$AWS_DEFAULT_REGION.amazonaws.com/$IMAGE_REPO_NAME:$IMAGE_TAG

  post_build:
    commands:
      - echo Sigularity build completed on `date`
      - echo Exporting $IMAGE_TAG.sif to artifacts 
artifacts:
  files:
    - /tmp/$IMAGE_TAG.sif

"""

### Step 6. Create an ECR repo
Before we can actually build our image, we need to have the repository referenced in our (post_build) phase. We will use boto3 again to interact with the AWS ECR APIs. We will actually use the repository in Step 6,  after the container image is built. 

In [ ]:
ecr_client = boto3.client('ecr')
try:
    resp = ecr_client.create_repository(repositoryName=proj_name)
except ClientError as e:
    if e.response['Error']['Code'] == 'RepositoryAlreadyExistsException':
        print(f"ECR Repo {proj_name} already exists, skip")

### Step 7. Create an AWS CodeCommit repo and checkin the files

We start by setting up the proper access permissions using the IAM service. Each service (CodePipeline, CodeBuild) needs its own policies. We also need to allow these services to access other related services on our behalf (S3 and ECR).

Note the CodePipeline and CodeBuild role ARNs that we will use in Step 6.

In [ ]:
# roleArn:
iam_client = session.client('iam')

codepipeline_service_role_name = f"{proj_name}-codepipeline-service-role"
codepipeline_policies = ['arn:aws:iam::aws:policy/AWSCodePipeline_FullAccess', 
                         'arn:aws:iam::aws:policy/AWSCodeCommitFullAccess',
                         'arn:aws:iam::aws:policy/AmazonS3FullAccess',
                         'arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess'
                        ]
codepipeline_role_arn = workshop.create_service_role_with_policies(codepipeline_service_role_name, 'codepipeline.amazonaws.com', codepipeline_policies )
print(codepipeline_role_arn)
              
codebuild_service_role_name = f"{proj_name}-codebuild-service-role"
codebuild_policies = ['arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess',
                      'arn:aws:iam::aws:policy/CloudWatchFullAccess',
                      'arn:aws:iam::aws:policy/AmazonS3FullAccess',
                      'arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess']
codebuild_role_arn = workshop.create_service_role_with_policies(codebuild_service_role_name, 'codebuild.amazonaws.com', codebuild_policies )
print(codebuild_role_arn)


Let's prepare our files for the initial checkin. We have the Dockerfile, sratest script, buildspec and filelist (empty for now).

In [ ]:
# prepare the files for the checkin
put_files=[{
               'filePath': 'Dockerfile.cicd',
               'fileContent': dockerfile_content
            },
            {
               'filePath': 'sratest.sh',
               'fileContent': sratest_content
            },
            {
               'filePath': 'filelist.txt',
               'fileContent': file_list_content
            },
            {
               'filePath': 'buildspec.yml',
               'fileContent': buildspec_content
            },
            {
               'filePath': 'buildspec_sig.yml',
               'fileContent': buildspec_content_sig
            }
        ]

We are now ready for the first commit. We will create our code repository and upload our files into the "main" branch.

In [ ]:

def check_in_files(proj_name, put_files):
    codecommit_client = boto3.client('codecommit')
    try:
        resp = codecommit_client.create_repository(repositoryName=proj_name)
    except ClientError as e:
        if e.response['Error']['Code'] == 'RepositoryNameExistsException':
            print(f"Repo {proj_name} exists, use that one")

    try:
        resp = codecommit_client.get_branch(repositoryName=proj_name, branchName='main')
        parent_commit_id = resp['branch']['commitId']
    except ClientError as e:
        if e.response['Error']['Code'] == 'BranchDoesNotExistException':
            # the repo is new, create it 
            workshop.commit_files(proj_name, "main", put_files,  None)
    else:
        try:
            resp = workshop.commit_files(proj_name, "main",put_files, parent_commit_id)
        except ClientError as ee:
            if ee.response['Error']['Code'] == 'NoChangeException':
                print('No change detected. skip commit')
                
check_in_files(proj_name, put_files)

### Step 8. Create a CodeBuild project

The second stage of the CI/CD pipeline is the build process. We use an instance managed by AWS (see computeType below) to build the container using a standard Amazon Linux 2 build environment. The CodeBuild process is triggered by the CodeCommit code checkins. 

Note: **codebuild-service-role takes a little longer to propagate**. If you see a permission error, please retry again in a minute.




In [ ]:
codebuild_client = boto3.client('codebuild')

codebuild_name = f"build-container" 
codecommit_name = f"Source-{proj_name}"
try: 
    resp = codebuild_client.create_project(name=codebuild_name, 
                                       description="CICD workshop build demo",
                                       source= {
                                           'type': "CODEPIPELINE"
                                       },
                                       artifacts= {
                                            "type": "CODEPIPELINE",
                                            "name": proj_name
                                       },
                                       environment= {
                                            "type": "LINUX_CONTAINER",
                                            "image": "aws/codebuild/amazonlinux2-x86_64-standard:3.0",
                                            "computeType": "BUILD_GENERAL1_SMALL",
                                            "environmentVariables": [
                                                {
                                                    "name": "AWS_DEFULT_REGION",
                                                    "value": region_name,
                                                    "type": "PLAINTEXT"
                                                },
                                                {
                                                    "name": "AWS_ACCOUNT_ID",
                                                    "value": account_id,
                                                    "type": "PLAINTEXT"
                                                },
                                                {
                                                    "name": "IMAGE_REPO_NAME",
                                                    "value": proj_name,
                                                    "type": "PLAINTEXT"
                                                },
                                                {
                                                    "name": "IMAGE_TAG",
                                                    "value": image_tag,
                                                    "type": "PLAINTEXT"
                                                }
                                            ],
                                            "privilegedMode": True,
                                            "imagePullCredentialsType": "CODEBUILD"               
                                       },
                                       logsConfig= {
                                                "cloudWatchLogs": {
                                                    "status": "ENABLED",
                                                    "groupName": proj_name
                                                },
                                                "s3Logs": {
                                                    "status": "DISABLED"
                                                }
                                        },
                                        serviceRole= codebuild_role_arn
                                      )
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceAlreadyExistsException':
        print(f"CodeBuild project {proj_name} exists, skip...")
    else:
        raise e


print(f"CodeBuild project name {codebuild_name}")

In [ ]:
codebuild_name_sig = f"build-singularity" 
try: 
    resp = codebuild_client.create_project(
                                       name=codebuild_name_sig, 
                                       description="Build Singularity Container",
                                       source= {
                                           'type': "CODEPIPELINE",
                                           'buildspec': "buildspec_sig.yml"
                                       },
                                       artifacts= {
                                            "type": "CODEPIPELINE",
                                            "name": proj_name
                                       },
                                       environment= {
                                            "type": "LINUX_CONTAINER",
                                            "image": "aws/codebuild/amazonlinux2-x86_64-standard:3.0",
                                            "computeType": "BUILD_GENERAL1_SMALL",
                                            "environmentVariables": [
                                                {
                                                    "name": "AWS_DEFULT_REGION",
                                                    "value": region_name,
                                                    "type": "PLAINTEXT"
                                                },
                                                {
                                                    "name": "AWS_ACCOUNT_ID",
                                                    "value": account_id,
                                                    "type": "PLAINTEXT"
                                                },
                                                {
                                                    "name": "IMAGE_REPO_NAME",
                                                    "value": proj_name,
                                                    "type": "PLAINTEXT"
                                                },
                                                {
                                                    "name": "IMAGE_TAG",
                                                    "value": image_tag,
                                                    "type": "PLAINTEXT"
                                                }
                                            ],
                                            "privilegedMode": True,
                                            "imagePullCredentialsType": "CODEBUILD"               
                                       },
                                       logsConfig= {
                                                "cloudWatchLogs": {
                                                    "status": "ENABLED",
                                                    "groupName": proj_name
                                                },
                                                "s3Logs": {
                                                    "status": "DISABLED"
                                                }
                                        },
                                        serviceRole= codebuild_role_arn
                                      )
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceAlreadyExistsException':
        print(f"CodeBuild project {proj_name} exists, skip...")
    else:
        raise e


print(f"CodeBuild project name {codebuild_name}")

### Step 9. Build the AWS CodePipeline 

We now combine source, docker build, singularity build steps together into a pipeline with 4 stages commit and build. 

In [ ]:
codepipeline_client = boto3.client('codepipeline')

stage1 = {
    "name":f"{codecommit_name}",
    "actions": [
        {
            "name": "Source",
            "actionTypeId": {
                "category": "Source",
                "owner": "AWS",
                "provider": "CodeCommit",
                "version": "1"
            },
            "runOrder": 1,
            "configuration": {
                "BranchName": "main",
                "OutputArtifactFormat": "CODE_ZIP",
                "PollForSourceChanges": "true",
                "RepositoryName": proj_name
            },
            "outputArtifacts": [
                {
                    "name": "SourceArtifact"
                }
            ],
            "inputArtifacts": [],
            "region": region_name,
            "namespace": "SourceVariables"
        }
    ]
}

stage2 = {
   "name": f"{codebuild_name}",
    "actions": [
        {
            "name": "BuildContainer",
            "actionTypeId": {
                "category": "Build",
                "owner": "AWS",
                "provider": "CodeBuild",
                "version": "1"
            },
            "runOrder": 1,
            "configuration": {
                "ProjectName": codebuild_name
            },
            "outputArtifacts": [
                {
                    "name": "BuildContainerArtifact"
                }
            ],
            "inputArtifacts": [
                {
                    "name": "SourceArtifact"
                }
            ],
            "region": region_name,
            "namespace": "BuildVariables"
        }
    ]    
}

stage3 = {
    "name": "Approval",
    "actions": [
        {
            "name": "ApproveMoveToBuildSingularity",
            "actionTypeId": {
                "category": "Approval",
                "owner": "AWS",
                "version": "1",
                "provider": "Manual"
            },
            "inputArtifacts": [],
            "outputArtifacts": [],
            "configuration": {
            #    "NotificationArn": "arn:aws:sns:us-east-2:80398EXAMPLE:MyApprovalTopic",
                "ExternalEntityLink": "https://us-east-1.console.aws.amazon.com/ecr/repositories?region=us-east-1",
                "CustomData": "Please review scan results in ECR repo"},
            "runOrder": 1
        }
    ]
}

stage4 = {
   "name": f"{codebuild_name_sig}",
    "actions": [
        {
            "name": "BuildSingularity",
            "actionTypeId": {
                "category": "Build",
                "owner": "AWS",
                "provider": "CodeBuild",
                "version": "1"
            },
            "runOrder": 1,
            "configuration": {
                "ProjectName": codebuild_name_sig
            },
            "outputArtifacts": [
                {
                    "name": "BuildSingularityArtifact"
                }
            ],
            "inputArtifacts": [
                {
                    "name": "SourceArtifact"
                }
            ],
            "region": region_name,
            "namespace": "BuildSingularityVariables"
        }
    ]
}

stages = [stage1, stage2, stage3, stage4]


pipeline = {
    'name': proj_name,
    'roleArn': codepipeline_role_arn,
    'artifactStore': {
        'type': 'S3',
        'location': bucket
    }, 
    'stages': stages
}

try:
    print("Creating pipeline")
    resp = codepipeline_client.create_pipeline( pipeline= pipeline)
    print("Created pipeline",resp)
except ClientError as e:
    print(e)
    if e.response['Error']['Code'] == 'PipelineNameInUseException':
        print(f"Codepipeline {proj_name} already exists " )

#### Note:

After the pipeline is created, please switch to the Code Pipeline console, you will see that the pipeline is running 

https://us-east-1.console.aws.amazon.com/codesuite/codepipeline/pipelines/sigularity-cicd-pipeline/view?region=us-east-1

When stage 2 is completed, you will need to approve the 

### Step 8. Check the container image in the repo

Navigate to CodePipeline in the AWS Console to check the status of the step above. The initial CodePipline process will take a few minutes. It will pull assets from CodeCommit, build the docker image on a managed instance, and push the result image into ECR. 

In [ ]:
# We should see a container image with the image tag "mySRATools" - this is defined as an environment variable in CodeBuild
#resp = ecr_client.list_images(repositoryName=proj_name)
while True:
    resp = ecr_client.describe_images(repositoryName=proj_name)
    if resp['imageDetails']:
        for image in resp['imageDetails']:
            print("image pushed at: " + str(image['imagePushedAt']))
        break
    else:
        clear_output(wait=True)
        display("Build not done yet, please wait and retry this step. Please do not proceed until you see the 'image pushed' message")
        time.sleep(20)
# this is used later in job_definition for AWS Batch
image_uri= f"{account_id}.dkr.ecr.{region_name}.amazonaws.com/{proj_name}:{image_tag}"
print(image_uri)


### Step 9. Now lets update the Dockerfile or the program files

just simply add an echo command to the strtest_content script.  The commit step won't run if no files are updated. 

In [ ]:
dockerfile_content="""FROM public.ecr.aws/ubuntu/ubuntu:latest

RUN apt-get update 
RUN DEBIAN_FRONTEND="noninteractive" apt-get -y install tzdata 
RUN apt-get install -y curl wget libxml-libxml-perl awscli uuid-runtime
        
RUN wget -q https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current-ubuntu64.tar.gz -O /tmp/sratoolkit.tar.gz \
        && tar zxf /tmp/sratoolkit.tar.gz -C /opt/ && rm /tmp/sratoolkit.tar.gz && \
        ln -s /opt/sratoolkit.$(curl -s "https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current.version")-ubuntu64 /opt/sratoolkit
        
ENV PATH="/opt/sratoolkit/bin/:${PATH}"
ADD sratest.sh /usr/local/bin/sratest.sh
RUN chmod +x /usr/local/bin/sratest.sh
RUN mkdir /tmp/.ncbi && printf '/LIBS/GUID = "%s"\\n' `uuidgen` > /tmp/.ncbi/user-settings.mkfg

ADD filelist.txt /tmp/filelist.txt
ENV HOME=/tmp
WORKDIR /tmp
USER nobody
ENTRYPOINT ["/usr/local/bin/sratest.sh"]
"""

sratest_content = """#!/bin/bash
set -x

prefetch $PACKAGE_NAME --output-directory /tmp
fasterq-dump $PACKAGE_NAME -e 8
echo 'done - upload files'
aws s3 sync . $SRA_OUTPUT/$PACKAGE_NAME
"""

put_files=[{
               'filePath': 'Dockerfile.cicd',
               'fileContent': dockerfile_content
            },
            {
               'filePath': 'sratest.sh',
               'fileContent': sratest_content
            },
            {
               'filePath': 'filelist.txt',
               'fileContent': file_list_content
            },
            {
               'filePath': 'buildspec.yml',
               'fileContent': buildspec_content
            },
            {
               'filePath': 'buildspec_sig.yml',
               'fileContent': buildspec_content_sig
            }
        ]


check_in_files(proj_name, put_files)

### Check the stage2 output 

In our example, the output artifact is stored as "/tmp/mySRCTool.sif" , but the artifact from the build pipeline is actually a zipped file with a shortname 

when you query the execution details, your will see something like 

```
Container output: {'bucket': 'container-sigularity-cicd-pipeline-xxxxx', 'key': 'sigularity-cicd-pipe/BuildSingu/XASsVIM'}
```

In [ ]:
import zipfile

resp = codepipeline_client.list_action_executions(pipelineName=proj_name)

s3 = boto3.client('s3')


if resp['actionExecutionDetails']:
    for ae in resp['actionExecutionDetails']:
        sig_image_location = ae['output']['outputArtifacts'][0]['s3location']
        print("Container output: " + str(sig_image_location))
        s3.download_file(sig_image_location['bucket'], sig_image_location['key'], f"{image_tag}.zip")
        
        print(f"Extract the image into ./tmp/image_tag.sif") 
        
        with zipfile.ZipFile(f"{image_tag}.zip", 'r') as zip_file: 
            zip_file.extractall(".")
        #only interested in the latest one
        break
else:
    display("No pipeline execution details")


# Don't forget to clean up 

Only do this if you want to delete your pipeline, and related resources

In [ ]:

# delete the CI/CD pipeline and repository
codepipeline_client.delete_pipeline(name=proj_name)
codebuild_client.delete_project(name=codebuild_name)
codebuild_client.delete_project(name=codebuild_name_sig)
codecommit_client.delete_repository(repositoryName=proj_name)
workshop.delete_codecommit_repo(proj_name)
workshop.delete_ecr_repo(proj_name)
workshop.delete_service_role_with_policies(codepipeline_service_role_name, codepipeline_policies )
workshop.delete_service_role_with_policies(codebuild_service_role_name, codebuild_policies )



In [ ]:
# Cleanup S3 bucket
workshop.delete_bucket_with_version(bucket)

In [ ]:
try:
    resp = iam_client.delete_role_policy(RoleName=ROLE_NAME, PolicyName='S3AccessPolicy')
except:
    print("Policy might have been deleted already. Ignore")